In [ ]:
!pip install pypdf langchain langchain_openai langchain_community kiwipiepy rank_bm25

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "/content/drive/MyDrive/langchain-tutorial/Ch04. Advanced Rag/Data/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap = 200)

docs = loader.load_and_split(doc_splitter)

In [ ]:
from langchain_community.retrievers import BM25Retriever
from kiwipiepy import Kiwi

kiwi_tokenizer = Kiwi()

def kiwi_tokenize(text):
  return [token.form for token in kiwi_tokenizer.tokenize(text)]

In [ ]:
bm25_retriever = BM25Retriever.from_documents(docs, preprocess_func=kiwi_tokenize)
bm25_retriever.k = 2

In [ ]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

# 관련있는 문서를 수집 후, Chatgpt로 최종 답변까지 수행하는 체인을 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.2, model="gpt-4o"),
    chain_type="stuff",
    retriever=bm25_retriever,
    return_source_documents=True # 답변에 사용된 source document도 보여주도록 설정
)


In [ ]:
qa_chain.invoke("이 회사가 발행한 주식의 총 발행량이 어느정도야?")